# 이민정 week5 문제 풀이

torch import하기

In [108]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

Q1. 가장 먼저 학습 데이터를 준비해보도록 하겠습니다. MNIST 데이터셋을 직접 Load
해 봅시다. 데이터셋을 로드하고 DataLoader를 구현해보세요.

- DataLoader를 이용해 MNIST 데이터셋을 로드해봅시다.

In [102]:
training_epochs = 15 #training 반복 횟수
batch_size = 100

In [103]:
root = './data'

#transforms.ToTensor(): Convert a PIL(파이썬 이미지 분석 라이브러리) Image or numpy.ndarray to tensor

mnist_train = dsets.MNIST(root=root, train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dsets.MNIST(root=root, train=False, transform=transforms.ToTensor(), download=True)

train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, drop_last=True)

Q2.데이터가 준비가 되었다면, 이제 그 데이터를 학습할 모델을 구현할 차례입니다. 그 후 모델 안의 가중치를 초기화시켜보세요. 입력 데이터 형태에 맞도록 linear한 모델을 구성해보세요.

- MNIST 입력의 크기는 28x28입니다.

- 여기서 구현하는 linear 모델은 입력이 1차원이기 때문에 입력 차원을 맞춰보세요.

In [104]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
linear = torch.nn.Linear(28*28, 10, bias=True).to(device)

#weight init
torch.nn.init.normal_(linear.weight)

Parameter containing:
tensor([[-1.0262, -1.0838, -0.0223,  ..., -1.6097,  0.2824, -0.0643],
        [ 0.6493,  0.6819, -0.1329,  ..., -0.4401,  0.1927, -0.7319],
        [ 0.3524, -1.6678,  0.0317,  ..., -0.7048,  0.2957,  0.8456],
        ...,
        [ 0.1587,  0.7049,  0.4860,  ...,  0.7190,  1.2466, -0.4460],
        [ 0.1118, -0.0316,  0.4709,  ...,  2.0205, -0.6535,  1.6389],
        [ 1.0071, -0.4865,  0.7262,  ..., -0.0762, -1.7246,  0.0552]],
       requires_grad=True)

Q3. 위에서 구현한 모델을 학습시키기 위해서는 loss 함수와 opitmizer가 필요합니다. 아래 제시된 loss 함수와 optimizer를 구현해보세요. Loss 함수와 optimizer는 모델 안의 가중치를 업데이트 할 때 사용됩니다.

옵티마이저는 SGD, Loss는 Cross Entropy Loss를 사용합니다.

In [105]:
#loss fn - Cross Entropy Loss
criterion = torch.nn.CrossEntropyLoss().to(device)

#optimizer - SGD
optimizer = torch.optim.SGD(linear.parameters(), lr=0.1)

Q4. 3번 문제까지 해결하셨다면, 이제 학습을 위한 준비는 거의 끝났다고 볼 수 있습니다. 위 구현 함수들을 이용해 학습 Loop를 구현해보세요.

- 위에서 구현한 모델, optimzer, loss fn 등을 이용해 학습을 구현해주세요.

In [106]:
for epoch in range(training_epochs):
    for i, (imgs, labels) in enumerate(train_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = imgs.view(-1, 28*28)
        
        outputs = linear(imgs) #imgs를 linear layer의 입력으로 넣어 classifier가 분류한 결과를 얻는다.
        loss = criterion(outputs, labels) #classifier가 분류한 결과와 실제 label로 cross_entropy를 진행한다.
        
        optimizer.zero_grad() #gradient 초기화
        loss.backward() #gradient를 계산해서 업데이트
        optimizer.step() #개선
        
        _, argmax = torch.max(outputs, 1) #max()를 이용해 최종 출력이 가장 높은 class 선택
        accuracy = (labels == argmax).float().mean()
        
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch+1, training_epochs, i+1, len(train_loader), loss.item(), accuracy.item() * 100))

Epoch [1/15], Step [100/600], Loss: 3.6108, Accuracy: 43.00%
Epoch [1/15], Step [200/600], Loss: 1.9255, Accuracy: 59.00%
Epoch [1/15], Step [300/600], Loss: 1.9646, Accuracy: 68.00%
Epoch [1/15], Step [400/600], Loss: 1.7222, Accuracy: 71.00%
Epoch [1/15], Step [500/600], Loss: 1.3860, Accuracy: 75.00%
Epoch [1/15], Step [600/600], Loss: 0.8740, Accuracy: 76.00%
Epoch [2/15], Step [100/600], Loss: 0.9672, Accuracy: 74.00%
Epoch [2/15], Step [200/600], Loss: 0.8345, Accuracy: 78.00%
Epoch [2/15], Step [300/600], Loss: 0.8401, Accuracy: 79.00%
Epoch [2/15], Step [400/600], Loss: 1.1615, Accuracy: 84.00%
Epoch [2/15], Step [500/600], Loss: 0.9892, Accuracy: 76.00%
Epoch [2/15], Step [600/600], Loss: 0.6248, Accuracy: 84.00%
Epoch [3/15], Step [100/600], Loss: 0.7774, Accuracy: 79.00%
Epoch [3/15], Step [200/600], Loss: 0.5385, Accuracy: 80.00%
Epoch [3/15], Step [300/600], Loss: 1.0808, Accuracy: 81.00%
Epoch [3/15], Step [400/600], Loss: 1.4527, Accuracy: 74.00%
Epoch [3/15], Step [500/

Q5. 학습이 완료되면, 모델이 잘 동작하는지 테스트가 필요합니다. 데이터로드 파트에서 준비했던 테스트 데이터를 이용해 테스트를 진행해봅시다. 아래 테스트 코드를 완성해보세요.

In [107]:
linear.eval()

with torch.no_grad():
    correct = 0
    total = 0
    
    for i, (imgs, labels) in enumerate(test_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = imgs.view(-1, 28 * 28)
    
        outputs = linear(imgs)
        
        _, argmax = torch.max(outputs, 1) #max()를 이용해 최종 출력이 가장 높은 class 선택
        
        for label, argmax in zip(labels, argmax):
            if label == argmax: #label이 argmax와 일치하면 correct를 +1 해주기
                correct += 1
            total += 1 #반복문 돌 때마다 total +1 해주기
        
    print('Test accuracy for {} images: {:.2f}%'.format(total, correct / total * 100))

Test accuracy for 10000 images: 89.01%
